 1. Construa um grafo baseado nesse dataset. (tutoriais A1, A3)

In [1]:
from netpixi.integration.gt import Graph

g = Graph(directed=True) # dirigido

ModuleNotFoundError: No module named 'netpixi'

In [ ]:
# criando vertices
g.add_vertex('ego')
g.add_vertex('facebook')
g.add_vertex('twitter')
g.add_vertex('gplus')

In [ ]:
# criando arestas
g.add_edge('ego', 'facebook')
g.add_edge('ego', 'twitter')
g.add_edge('ego', 'gplus')